In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction'

In [8]:
import pandas as pd
pd.set_option("display.max_columns",None)

In [11]:
data = pd.read_csv("artifacts\\data_ingestion\\csv_result-bone-marrow.csv")

In [12]:
data.head()

,id,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,ABOmatch,CMVstatus,DonorCMV,RecipientCMV,Disease,Riskgroup,Txpostrelapse,Diseasegroup,HLAmatch,HLAmismatch,Antigen,Alel,HLAgrI,Recipientage,Recipientage10,Recipientageint,Relapse,aGvHDIIIIV,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
0,1,1,1,22.830137,0,1,0,1,1,1,0,3,1,1,ALL,1,0,1,0,0,-1,-1,0,9.6,0,1,0,0,1,7.20,1.33876,5.38,35,19,51,32,999,0
1,2,1,0,23.342466,0,1,0,-1,-1,1,0,0,0,0,ALL,0,0,1,0,0,-1,-1,0,4.0,0,0,1,1,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
2,3,1,0,26.394521,0,1,0,-1,-1,1,0,2,0,1,ALL,0,0,1,0,0,-1,-1,0,6.6,0,1,1,1,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
3,4,0,0,39.684932,1,1,0,1,2,1,1,1,1,0,AML,0,0,1,0,0,-1,-1,0,18.1,1,2,0,0,?,4.25,29.481647,0.14,50,23,29,19,53,1
4,5,0,1,33.358904,0,0,0,1,2,0,1,0,0,1,chronic,1,0,1,1,0,1,0,1,1.3,0,0,0,1,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    187 non-null    int64  
 1   Recipientgender       187 non-null    int64  
 2   Stemcellsource        187 non-null    int64  
 3   Donorage              187 non-null    float64
 4   Donorage35            187 non-null    int64  
 5   IIIV                  187 non-null    int64  
 6   Gendermatch           187 non-null    int64  
 7   DonorABO              187 non-null    int64  
 8   RecipientABO          187 non-null    object 
 9   RecipientRh           187 non-null    object 
 10  ABOmatch              187 non-null    object 
 11  CMVstatus             187 non-null    object 
 12  DonorCMV              187 non-null    object 
 13  RecipientCMV          187 non-null    object 
 14  Disease               187 non-null    object 
 15  Riskgroup             1

In [14]:
data.shape

(187, 38)

### entity

In [69]:
from pathlib import  Path
from dataclasses import dataclass

@dataclass
class DataValidationConfig:
    root_dir : Path
    STATUS_FILE : str
    unzip_dir : Path
    all_schema : dict
    


### Configuration manager

In [70]:
from Bone_marrow_survival_prediction.constants import *
from Bone_marrow_survival_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            unzip_dir = config.unzip_dir,
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            all_schema = schema
        )
        
        return data_validation_config
        

### coponent

In [71]:
import os
import pandas as pd
from Bone_marrow_survival_prediction import logger

class DataValidation:
    def __init__(self, config):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_dir)
            all_cols = list(data.columns)

            all_schema = list(self.config.all_schema.keys())

            for col in all_cols:
                if col not in all_schema and col != "survival_status":
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"validation_status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"validation_status: {validation_status}")

            return validation_status

        except Exception as e:
            logger.error(f"Error during validation: {e}")
            raise e


### Pipeline

In [72]:
try :
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[ 2025-01-18 21:15:23,402: INFO: comman: yaml file: config\config.yaml loaded successfully]
[ 2025-01-18 21:15:23,405: INFO: comman: yaml file: params.yaml loaded successfully]
[ 2025-01-18 21:15:23,409: INFO: comman: yaml file: schema.yaml loaded successfully]
[ 2025-01-18 21:15:23,411: INFO: comman: created directory at: artifacts]
[ 2025-01-18 21:15:23,414: INFO: comman: created directory at: artifacts/data_validation]
